In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score


In [53]:

df = pd.read_csv('donnees_nettoyees.csv')

# Vérifier les premières lignes pour s'assurer que tout est bon
df.head()


,State,NAICS,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,LowDoc,DisbursementGross,MIS_Status,GrAppv,SBA_Appv
0,0,451120,1997,84,4,2.0,0,0,1,0,1,60000.0,1,60000.0,48000.0
1,0,722410,1997,60,2,2.0,0,0,1,0,1,40000.0,1,40000.0,32000.0
2,0,621210,1997,180,7,1.0,0,0,1,0,0,287000.0,1,287000.0,215250.0
3,1,0,1997,60,2,1.0,0,0,1,0,1,35000.0,1,35000.0,28000.0
4,2,0,1997,240,14,1.0,7,7,1,0,0,229000.0,1,229000.0,229000.0


In [54]:
df["MIS_Status"].value_counts()

MIS_Status
1    741345
0    157819
Name: count, dtype: int64

In [55]:

# 2. Charger les données dans un DataFrame (remplace 'df' par ton DataFrame déjà chargé)
# Assurez-vous que 'MIS_Status' est la variable cible
# X : toutes les colonnes explicatives, sauf MIS_Status
# y : colonne MIS_Status

# 3. Encodage de y (si nécessaire)
# Si 'MIS_Status' est une chaîne de caractères, on le convertit en 0 et 1

X = df.drop(columns=['MIS_Status'])
y = df['MIS_Status']


In [56]:
y.isna().sum()

np.int64(0)

In [57]:
# 4. Séparation des données en jeu d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Construction et entraînement du modèle
model = LogisticRegression(max_iter=1000)  # max_iter pour éviter les problèmes de convergence
model.fit(X_train, y_train)

# 6. Prédictions sur les données de test
y_pred = model.predict(X_test)

# 7. Évaluation du modèle
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8325112743489793

Confusion Matrix:
 [[  4722  26701]
 [  3419 144991]]

Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.15      0.24     31423
           1       0.84      0.98      0.91    148410

    accuracy                           0.83    179833
   macro avg       0.71      0.56      0.57    179833
weighted avg       0.80      0.83      0.79    179833



/Users/khadijaabdelmalek/Documents/SIMPLON/BRIEF/Brief_classification/venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [60]:
f1macro = f1_score(y_test, y_pred, average='macro')
print("score f1 macro :" , f1macro)

score f1 macro : 0.5723034214751843


In [21]:
# Vérifier le type de chaque colonne
print(df.dtypes)


State                  int64
NAICS                  int64
ApprovalFY             int64
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
LowDoc                 int64
DisbursementGross    float64
MIS_Status             int64
GrAppv               float64
SBA_Appv             float64
dtype: object


In [22]:
# Vérifier les valeurs manquantes (NaN) dans chaque colonne
nan_count = df.isna().sum()

# Afficher les colonnes avec des NaN
print(nan_count[nan_count > 0])


Series([], dtype: int64)


In [23]:
df.isna().sum()


State                0
NAICS                0
ApprovalFY           0
Term                 0
NoEmp                0
NewExist             0
CreateJob            0
RetainedJob          0
FranchiseCode        0
UrbanRural           0
LowDoc               0
DisbursementGross    0
MIS_Status           0
GrAppv               0
SBA_Appv             0
dtype: int64

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Initialisation du modèle
dt_model = DecisionTreeClassifier(random_state=42)

# Entraînement
dt_model.fit(X_train, y_train)

# Prédiction
y_pred_dt = dt_model.predict(X_test)

# Évaluation
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))


ValueError: Input y contains NaN.

In [25]:
print(y.isnull().sum())


899164


In [62]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Charger les données
data = pd.read_csv("/mnt/data/donnees_nettoyees.csv")

# Séparer les features et la target
y = data['GrAppv']  # Variable cible (à adapter si besoin)
X = data.drop(['GrAppv'], axis=1)

# Définir les colonnes numériques et catégoriques
numeric_features = ["Term", "NoEmp", "CreateJob", "RetainedJob", "DisbursementGross", "SBA_Appv"]
categorical_features = ["State", "NAICS", "ApprovalFY", "NewExist", "FranchiseCode", "UrbanRural", "LowDoc", "MIS_Status"]

# Prétraitement : StandardScaler pour les numériques, OneHotEncoder pour les catégoriques
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Construire la pipeline avec CatBoostRegressor
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', CatBoostRegressor(verbose=0))
])

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner la pipeline
pipeline.fit(X_train, y_train)

# Évaluer le modèle
score = pipeline.score(X_test, y_test)
print(f"Score R² du modèle sur l'ensemble de test : {score:.2f}")


ModuleNotFoundError: No module named 'sklearn.pipeline'